### Categorize kanji

In [1]:
# imports
import pandas

from dataclasses import dataclass

# data classes
@dataclass        
class Kanji:
    rank: int
    name: str

In [2]:
# read excel file (with mapping)
df_kanji = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="MAIN")
df_kanji.columns = ["CHAR", "COMPONENTS1", "COMPONENTS2", "COMPONENTS3", "COMPONENTS4", "COMPONENTS5", "ON READING", "KUN READING", "KEYWORD", "SRL", "TYPE", "FREQ", "TAGS"]
df_kanji

,CHAR,COMPONENTS1,COMPONENTS2,COMPONENTS3,COMPONENTS4,COMPONENTS5,ON READING,KUN READING,KEYWORD,SRL,TYPE,FREQ,TAGS
0,一,NaN,NaN,NaN,NaN,NaN,イチ、イツ,ひと・つ,one,5,STEM,10.800000,NaN
1,二,NaN,NaN,NaN,NaN,NaN,ニ,ふた・つ,two,5,MEAN,128.300000,NaN
2,三,NaN,NaN,NaN,NaN,NaN,サン,みっ・つ,three,5,MEAN,120.700000,NaN
3,四,NaN,NaN,NaN,NaN,NaN,シ,よっ・つ、よん、よ,four,5,MEAN,312.073333,NaN
4,五,NaN,NaN,NaN,NaN,NaN,ゴ,いつ・つ,five,5,MEAN,315.626667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,気,气,㐅,NaN,NaN,NaN,キ、ケ,NaN,atmosphere,1,OTHER,64.000000,NaN
1496,風,𠘨,䖝,NaN,NaN,NaN,フウ、フ,かぜ,1 wind 2 style,1,OTHER,289.700000,NaN
1497,乳,⺤,子,乚,NaN,NaN,ニュウ,ち、ちち,milk,1,OTHER,1067.000000,NaN
1498,興,臼,同,ハ,NaN,NaN,コウ、キョウ,NaN,interest,1,OTHER,649.600000,NaN


In [3]:
df_keyword = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="keyword.list")
# df_keyword

In [4]:
df_stem = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="stem.list")
# df_stem

### Algorithm

In [55]:
random_row = df_kanji[df_kanji["CHAR"] == '由'].iloc[0]
random_row

CHAR                由
COMPONENTS1       NaN
COMPONENTS2       NaN
COMPONENTS3       NaN
COMPONENTS4       NaN
COMPONENTS5       NaN
ON READING       ユ、ユウ
KUN READING       NaN
KEYWORD        reason
SRL                 1
TYPE               VR
FREQ            364.3
TAGS              NaN
Name: 243, dtype: object

In [76]:
categorization = {}
special_grp = '78 special'
other_grp = '77 other'

for grp in df_keyword["GROUP"].unique():
    categorization[grp] = []

for grp in df_stem["GROUP"].unique():
    categorization[grp] = []

categorization[other_grp] = []
categorization[special_grp] = []

# categorization["1 numbers"] = []
# categorization["1 numbers"].append(Test("aa", "bb"))
# categorization["1 numbers"].append(Test("ca", "ab"))
# categorization["1 numbers"].insert(0, Test("xxx", "ab"))

# categorization

queue_categorization = {}
queue_categorization

{}

In [79]:
# first rule (check keyword)
def find_keyword(row):
    group = df_keyword[df_keyword["KEYWORD"] == row["KEYWORD"]]["GROUP"]
    if group.empty:
        return "none"
    else:
        return group.iloc[0]

def find_stem(row):
    group = df_stem[df_stem["STEM KANJI"] == row["CHAR"]]["GROUP"]
    if group.empty:
        return "none"
    else:
        return group.iloc[0]

first_rule = find_keyword(random_row)
second_rule = find_stem(random_row)
if (first_rule != "none"):
    print("1. rule")
    if (random_row["TYPE"] == "MEAN"):
        categorization[first_rule].append(random_row)
    else:
        if (random_row["TYPE"] == "SPECIAL"):
            categorization[special_grp].append(random_row)
        else:
            if (random_row["TYPE"] == "OTHER"):
                categorization[other_grp].append(random_row)
            else:
                print("ERROR: missing grp")
else:
    if (second_rule != "none"):
        print("2. rule")
        if (random_row["TYPE"] == "STEM"):
            categorization[second_rule].insert(0, random_row)
        else:
            print("ERROR: missing rule")
    else:
        components = df_kanji[(df_kanji["COMPONENTS1"] == random_row["CHAR"]) | (df_kanji["COMPONENTS2"] == random_row["CHAR"])]
        print("components count: " + str(len(components.index)))
        if components.empty:
            print("nope")
        else:
            print("3. rule")
            vr_crowns = random_row["ON READING"].split("、")
            print(vr_crowns)
            onyomi = components[components["ON READING"].isin(vr_crowns)]
            if onyomi.empty:
                print("4. rule")
            else:
                print("3. rule a) b)")
                if len(onyomi.index) > 1:
                    print("kanji > 1")
                    print(onyomi)
                    max_srl_kanji = onyomi[onyomi["SRL"] == onyomi["SRL"].max()].iloc[0]
                    random_row["TAG"] = "CROWN_TAG"
                    random_row["TYPE"] = "VR"
                    
                    if max_srl_kanji["CHAR"] in queue_categorization.keys():
                        queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
                    else:
                        queue_categorization[max_srl_kanji["CHAR"]] = []
                        queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
                else:
                    print("kanji = 1")
                    random_row["TAG"] = "CROWN_TAG"
                    random_row["TYPE"] = "VR"
                    if max_srl_kanji["CHAR"] in queue_categorization.keys():
                        queue_categorization[onyomi.iloc[0]["CHAR"]].append(random_row)
                    else:
                        queue_categorization[onyomi.iloc[0]["CHAR"]] = []
                        queue_categorization[onyomi.iloc[0]["CHAR"]].append(random_row)
                    

                    
categorization
queue_categorization
# find_keyword(random_row)
# find_stem(random_row)

components count: 6
3. rule
['ユ', 'ユウ']
3. rule a) b)
kanji = 1


{'油': [CHAR                   由
  COMPONENTS1          NaN
  COMPONENTS2          NaN
  COMPONENTS3          NaN
  COMPONENTS4          NaN
  COMPONENTS5          NaN
  ON READING          ユ、ユウ
  KUN READING          NaN
  KEYWORD           reason
  SRL                    1
  TYPE                  VR
  FREQ               364.3
  TAGS                 NaN
  TAG            CROWN_TAG
  Name: 243, dtype: object,
  CHAR                   由
  COMPONENTS1          NaN
  COMPONENTS2          NaN
  COMPONENTS3          NaN
  COMPONENTS4          NaN
  COMPONENTS5          NaN
  ON READING          ユ、ユウ
  KUN READING          NaN
  KEYWORD           reason
  SRL                    1
  TYPE                  VR
  FREQ               364.3
  TAGS                 NaN
  TAG            CROWN_TAG
  Name: 243, dtype: object,
  CHAR                   由
  COMPONENTS1          NaN
  COMPONENTS2          NaN
  COMPONENTS3          NaN
  COMPONENTS4          NaN
  COMPONENTS5          NaN
  ON READING         

In [ ]:
# df[df["COMPONENTS1"] == "c"]
# mb = mb.dropna(subset=['ON READING'])
# mb[mb["ON READING"].str.ns("フク")]